# Failure Rate Calculation

Reproduce book calculations related to failure rates

In [1]:
# Check if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

# Set up helper functions
if IN_COLAB:
    # Download helper functions
    !wget -q https://raw.githubusercontent.com/yourusername/my-earthquake-book/main/helpers/utils.py
    !wget -q https://raw.githubusercontent.com/yourusername/my-earthquake-book/main/helpers/data_utils.py
    
    # Download any required data files
    !wget -q https://raw.githubusercontent.com/yourusername/my-earthquake-book/main/data/sample_data.csv

# Import helper functions
# from utils import calculate_fragility, plot_fragility_function

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm # type: ignore

# Set up color and line style specifications
colors = {
    1: np.array([56, 95, 150])/255,
    2: np.array([207, 89, 33])/255,
    3: np.array([158, 184, 219])/255,
    4: np.array([231, 184, 0])/255,
    5: np.array([128, 0, 0])/255
}

Specify a hazard curve and fragility parameters

In [ ]:
# load hazard curve (from simplified logic tree example)
hazard_curve = pd.read_csv('../data/hazard_curve_1.csv', dtype={'im': float, 'lambda_im': float})


# Define fragility function parameters
theta_im = 0.5  # Fragility median
beta_im = 0.4   # Fragility beta (standard deviation of the natural log of the IM)

Calculate and plot the fragility function

In [ ]:
# Calculate the fragility function (probability of failure given IM)
im = hazard_curve['im'].values
lambda_im = hazard_curve['lambda_im'].values
p_fail = norm.cdf(np.log(im), np.log(theta_im), beta_im)

# Plot the fragility function
plt.figure()
plt.plot(im, p_fail, '-', color=colors[1], linewidth=2)
plt.plot([0.01, theta_im, theta_im], [0.5, 0.5, 0], 'k:', linewidth=1)
plt.xlabel('Intensity Measure, IM')
plt.ylabel('P(F | IM = im)')
plt.axis([0, 1.5, 0, 1])
plt.grid(alpha=0.3)
plt.show()

Failure rate calculations

In [ ]:
# Calculate failure rate
# Take the absolute value of the negative derivative of the hazard curve
d_lambda = np.abs(np.append(np.diff(lambda_im), 0))
fail_contrib = p_fail * d_lambda
fail_rate = np.sum(fail_contrib)
print(f"Failure Rate: {fail_rate:.6e}")

# Plot hazard and fragility on the same plot using two y-axes
fig, ax1 = plt.subplots()

# Left y-axis: fragility function
ax1.set_xlabel('SA(1 s) [g]')
ax1.set_ylabel('Failure probability, P(F | IM = x)')
ax1.plot(im, p_fail, '-', color=colors[1], linewidth=2, label='Fragility, θ = 0.5 g')
ax1.set_xlim(0, 1.5)
ax1.set_ylim(0, 1)
ax1.tick_params(axis='y', labelcolor='black')

# Right y-axis: hazard curve
ax2 = ax1.twinx()
ax2.set_ylabel('Annual rate of exceedance, λ(IM>x)')
ax2.plot(im, lambda_im, '-', color=colors[3], linewidth=2, label='Ground-motion hazard')
ax2.set_ylim(0, 0.002)
ax2.tick_params(axis='y', labelcolor='black')

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower right')

plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Discretize for tabular output
x_short = np.arange(0.01, 1.51, 0.01)
lambda_short = np.interp(x_short, im, lambda_im)
d_lambda_short = np.abs(np.append(np.diff(lambda_short), lambda_short[-1]))
p_fail_short = norm.cdf(np.log(x_short), np.log(theta_im), beta_im)

fail_rate_discrete = np.sum(p_fail_short * d_lambda_short)
print(f"Failure Rate (discrete): {fail_rate_discrete:.6e}")

# Plot the failure rate contributions as a bar chart
plt.figure()
plt.bar(x_short, p_fail_short * d_lambda_short, width=0.008, color=colors[3], alpha=0.7)
plt.xlabel('SA(1 s) [g]')
plt.ylabel('$P(F | IM = x_i) Δλ_i$')
plt.xlim(0, 1.6)
plt.ylim(0, 8e-6)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Optional: Display the contributions for selected IM levels in a table
import pandas as pd

# Create a DataFrame for selected IM levels for demonstration
selected_indices = np.arange(0, len(x_short), 10)  # Every 10th value
selected_im = x_short[selected_indices]
selected_p_fail = p_fail_short[selected_indices]
selected_d_lambda = d_lambda_short[selected_indices]
selected_contrib = selected_p_fail * selected_d_lambda

df = pd.DataFrame({
    'IM': selected_im,
    'P(F|IM)': selected_p_fail,
    'ΔλIM': selected_d_lambda,
    'Contribution': selected_contrib
})

print("Selected IM level contributions to failure rate:")
display(df)